In [2]:
# Install Hugging Face datasets
!pip install datasets scikit-learn --quiet
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from datasets import load_dataset
import os
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cud

## UPLOAD DATASET TO GOOGLE DRIVE


In [4]:
# Install Hugging Face datasets
!pip install datasets scikit-learn --quiet

# Import required libraries
from datasets import load_dataset
import os
import shutil
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split

# Load the dataset
dataset = load_dataset("cats_vs_dogs")
full_dataset = dataset["train"]

# Extract labels
labels = full_dataset["labels"]

# Create stratified train/test indices using scikit-learn
train_indices, test_indices = train_test_split(
    np.arange(len(full_dataset)),
    test_size=0.1,
    random_state=42,
    stratify=labels
)

# Create train and test splits
train_dataset = full_dataset.select(train_indices)
test_dataset = full_dataset.select(test_indices)

# Verify the splits
print(f"Training examples: {len(train_dataset)}")
print(f"Testing examples: {len(test_dataset)}")

# Count class distribution in splits
train_cats = sum(1 for example in train_dataset if example["labels"] == 0)
train_dogs = sum(1 for example in train_dataset if example["labels"] == 1)
test_cats = sum(1 for example in test_dataset if example["labels"] == 0)
test_dogs = sum(1 for example in test_dataset if example["labels"] == 1)

print(f"Training set: {train_cats} cats, {train_dogs} dogs")
print(f"Testing set: {test_cats} cats, {test_dogs} dogs")

# Set up directories
save_dir = "/content/drive/MyDrive/cats_vs_dogs"

# Train folders
train_cats_dir = os.path.join(save_dir, "train", "cats")
train_dogs_dir = os.path.join(save_dir, "train", "dogs")

# Test folders
test_cats_dir = os.path.join(save_dir, "test", "cats")
test_dogs_dir = os.path.join(save_dir, "test", "dogs")

# Clear existing directories and create new ones
for directory in [train_cats_dir, train_dogs_dir, test_cats_dir, test_dogs_dir]:
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory, exist_ok=True)

# Save training images
train_cat_counter = 0
train_dog_counter = 0
print("Saving training images...")
for example in tqdm(train_dataset, desc="Saving train images"):
    if example["labels"] == 0:  # Cat
        filename = f"cat_{train_cat_counter}.jpg"
        path = os.path.join(train_cats_dir, filename)
        train_cat_counter += 1
    else:  # Dog
        filename = f"dog_{train_dog_counter}.jpg"
        path = os.path.join(train_dogs_dir, filename)
        train_dog_counter += 1
    example["image"].save(path)

print(f"Saved {train_cat_counter} cats and {train_dog_counter} dogs in training set")

# Save testing images
test_cat_counter = 0
test_dog_counter = 0
print("Saving testing images...")
for example in tqdm(test_dataset, desc="Saving test images"):
    if example["labels"] == 0:  # Cat
        filename = f"cat_{test_cat_counter}.jpg"
        path = os.path.join(test_cats_dir, filename)
        test_cat_counter += 1
    else:  # Dog
        filename = f"dog_{test_dog_counter}.jpg"
        path = os.path.join(test_dogs_dir, filename)
        test_dog_counter += 1
    example["image"].save(path)

print(f"Saved {test_cat_counter} cats and {test_dog_counter} dogs in test set")

# Verify that all directories have images
for directory in [train_cats_dir, train_dogs_dir, test_cats_dir, test_dogs_dir]:
    file_count = len(os.listdir(directory))
    print(f"{directory}: {file_count} images")

Training examples: 21069
Testing examples: 2341
Training set: 10567 cats, 10502 dogs
Testing set: 1174 cats, 1167 dogs
Saving training images...


Saving train images: 100%|██████████| 21069/21069 [06:34<00:00, 53.44it/s]


Saved 10567 cats and 10502 dogs in training set
Saving testing images...


Saving test images: 100%|██████████| 2341/2341 [00:43<00:00, 53.60it/s]


Saved 1174 cats and 1167 dogs in test set
/content/drive/MyDrive/cats_vs_dogs/train/cats: 10567 images
/content/drive/MyDrive/cats_vs_dogs/train/dogs: 10502 images
/content/drive/MyDrive/cats_vs_dogs/test/cats: 1174 images
/content/drive/MyDrive/cats_vs_dogs/test/dogs: 1167 images


## LOAD DATASET FROM GOOGLE DRIVE

In [5]:


# Path to your dataset
save_dir = "/content/drive/MyDrive/cats_vs_dogs"
cats_train_dir = os.path.join(save_dir, "train/cats")
dogs_train_dir = os.path.join(save_dir, "train/dogs")
cats_test_dir = os.path.join(save_dir, "test/cats")
dogs_test_dir = os.path.join(save_dir, "test/dogs")


# Count number of files (images) in each folder
num_train_cats = len([name for name in os.listdir(cats_train_dir) if os.path.isfile(os.path.join(cats_train_dir, name))])
num_train_dogs = len([name for name in os.listdir(dogs_train_dir) if os.path.isfile(os.path.join(dogs_train_dir, name))])
num_test_cats = len([name for name in os.listdir(cats_test_dir) if os.path.isfile(os.path.join(cats_test_dir, name))])
num_test_dogs = len([name for name in os.listdir(dogs_test_dir) if os.path.isfile(os.path.join(dogs_test_dir, name))])

print(f"Number of train cat images: {num_train_cats}")
print(f"Number of train dog images: {num_train_dogs}")
print(f"Number of test cat images: {num_test_cats}")
print(f"Number of test dog images: {num_test_dogs}")


Number of train cat images: 10567
Number of train dog images: 10502
Number of test cat images: 1174
Number of test dog images: 1167


## Model creation

In [6]:
model=Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100,100,3)),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
#Define loss function
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


## LOADING DATA to current workspace


In [8]:
import shutil
# Your current dataset path in Drive
drive_dataset_path = "/content/drive/MyDrive/cats_vs_dogs"

# Local dataset path in Colab
local_dataset_path = "/content/cats_vs_dogs"

# If local folder exists already, remove it first to avoid mixing old files
if os.path.exists(local_dataset_path):
    shutil.rmtree(local_dataset_path)

# Now copy the entire dataset from Drive to Colab
shutil.copytree(drive_dataset_path, local_dataset_path)

print("✅ Dataset copied to local Colab storage!")

# Verify structure
for root, dirs, files in os.walk(local_dataset_path):
    level = root.replace(local_dataset_path, '').count(os.sep)
    indent = ' ' * 2 * (level)
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for f in files[:5]:  # Only show first 5 files per folder
        print(f"{subindent}{f}")


✅ Dataset copied to local Colab storage!
cats_vs_dogs/
  test/
    dogs/
      dog_123.jpg
      dog_588.jpg
      dog_543.jpg
      dog_994.jpg
      dog_875.jpg
    cats/
      cat_532.jpg
      cat_277.jpg
      cat_1159.jpg
      cat_75.jpg
      cat_2.jpg
  train/
    dogs/
      dog_10403.jpg
      dog_4148.jpg
      dog_3889.jpg
      dog_5189.jpg
      dog_6599.jpg
    cats/
      cat_6128.jpg
      cat_6834.jpg
      cat_5489.jpg
      cat_6799.jpg
      cat_10041.jpg


##Training

In [ ]:

# Set up ImageDataGenerator for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,         # Normalize pixel values
    validation_split=0.2    # 20% of training data for validation
)
save_dir = "/content/cats_vs_dogs"
# Load training data
train_generator = train_datagen.flow_from_directory(
    os.path.join(save_dir, "train"),
    target_size=(100, 100),     # Match your CNN input size
    batch_size=32,
    class_mode='binary',        # Binary classification (cat vs dog)
    subset='training'
)

# Load validation data
validation_generator = train_datagen.flow_from_directory(
    os.path.join(save_dir, "train"),
    target_size=(100, 100),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Now fit the model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator
)
model_save_path = '/content/cats_vs_dogs_model.h5'
model.save(model_save_path)
print(f"✅ Model saved at: {model_save_path}")

Found 16856 images belonging to 2 classes.
Found 4213 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
214/527 ━━━━━━━━━━━━━━━━━━━━ 2:41 516ms/step - accuracy: 0.5577 - loss: 0.6934

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 1. Plot training/validation accuracy and loss curves with annotations
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(len(acc))

    plt.figure(figsize=(16, 6))

    # Accuracy Plot
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy', marker='o')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy', marker='x')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    # Annotate best validation accuracy
    best_epoch_acc = np.argmax(val_acc)
    best_val_acc = val_acc[best_epoch_acc]
    plt.scatter(best_epoch_acc, best_val_acc, color='red')
    plt.text(best_epoch_acc, best_val_acc + 0.01, f'{best_val_acc:.2f}', ha='center', color='red')

    # Loss Plot
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss', marker='o')
    plt.plot(epochs_range, val_loss, label='Validation Loss', marker='x')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')

    # Annotate minimum validation loss
    best_epoch_loss = np.argmin(val_loss)
    best_val_loss = val_loss[best_epoch_loss]
    plt.scatter(best_epoch_loss, best_val_loss, color='green')
    plt.text(best_epoch_loss, best_val_loss + 0.02, f'{best_val_loss:.2f}', ha='center', color='green')

    plt.tight_layout()
    plt.show()

# Then just call it:
plot_history(history)
